In [ ]:
import sys
import os
import torch
import numpy as np
current_directory = os.getcwd()
parent_directory = os.path.dirname(current_directory)
sys.path.append(parent_directory)
from ssast_model import ASTModel
import soundfile as sf
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import torchaudio
import pickle
import librosa

## reconstruct spectrogram

### function to reconstruct an audio signal from a spectrogram

In [ ]:
def synthesize_audio_from_spectrogram(torchaudio_spectrogram_to_synthesize, reference_audio_signal=None, sample_rate=16000):
    
    def fit_polynomial_transform(M1, M2, order=5):
        b = M1 - M2
        f_bins, t_frames = M1.shape
        f = np.repeat(np.arange(f_bins), t_frames)
        # Create polynomial features up to the given order
        A = np.column_stack([f**i for i in range(order + 1)])
        b = b.flatten()
        A_transpose = A.T
        A_transpose_A = np.dot(A_transpose, A)
        A_transpose_b = np.dot(A_transpose, b)
        x = np.linalg.solve(A_transpose_A, A_transpose_b)
        return x  # coefficients of the polynomial

    def mel_2_audio(M):
        sr = sample_rate
        fmin = 20
        n_fft = 512
        hop_length = int(0.01 * sr)  # 10 ms
        win_length = int(0.025 * sr)  # 25 ms 
        window = 'hanning'
        linear_M = librosa.db_to_power(M)
        audio = librosa.feature.inverse.mel_to_audio(linear_M, sr=sr, n_fft=n_fft, hop_length=hop_length, win_length=win_length, window=window, n_iter=50, htk=True, fmin=fmin)
        return audio
    
    if reference_audio_signal is not None:

        librosa_spectrogram_linear = librosa.feature.melspectrogram(
            y=reference_audio_signal,
            sr=sample_rate,
            n_mels=128,
            hop_length=160,  # 10 ms = 160 * 1/16000
            win_length=400,  # 25 ms = 400 * 1/16000
            n_fft=512,
            center=False,
            htk=True,
            fmin=20,
            fmax=sample_rate / 2  # default Nyquist frequency
        )

        reference_audio_tensor = torch.from_numpy(reference_audio_signal).float().unsqueeze(0)

        fbanks_to_fit = torchaudio.compliance.kaldi.fbank(
            reference_audio_tensor,
            sample_frequency=sample_rate,
            htk_compat=True,
            use_energy=False,
            window_type='hanning',
            num_mel_bins=128,
            dither=0.0,
            frame_shift=10
        )
        # now shape is torch.Size([time_frames, mel_frequency_bins])

        torchaudio_spectrogram_to_fit = fbanks_to_fit.cpu().numpy().T
        # now shape is (mel_frequency_bins, time_frames)

        torchaudio_spectrogram_to_fit_linear = np.exp(torchaudio_spectrogram_to_fit) # torchaudio uses a natural logarithm
        torchaudio_spectrogram_to_synthesize_linear = np.exp(torchaudio_spectrogram_to_synthesize) # torchaudio uses a natural logarithm

        torchaudio_spectrogram_to_fit_db = librosa.power_to_db(torchaudio_spectrogram_to_fit_linear)
        torchaudio_spectrogram_to_synthesize_db = librosa.power_to_db(torchaudio_spectrogram_to_synthesize_linear)
        librosa_spectrogram_linear_db = librosa.power_to_db(librosa_spectrogram_linear)

        # Calculate the polynomial transformation parameters
        coefficients = fit_polynomial_transform(np.array(librosa_spectrogram_linear_db), np.array(torchaudio_spectrogram_to_fit_db[:, :-1]))

        # Apply the polynomial transformation to each frequency bin across all time frames
        frequency_indices = np.arange(torchaudio_spectrogram_to_synthesize_db.shape[0]).reshape(-1, 1)
        adjustments = sum([coefficients[i] * frequency_indices**i for i in range(len(coefficients))])
        adjustments = np.tile(adjustments, (1, torchaudio_spectrogram_to_synthesize_db.shape[1]))
        adjusted_spectrogram_db = torchaudio_spectrogram_to_synthesize_db + adjustments

        # Resynthesize the audio from the adjusted spectrogram
        audio = mel_2_audio(adjusted_spectrogram_db)
    
    else:
        
        torchaudio_spectrogram_to_synthesize_linear = np.exp(torchaudio_spectrogram_to_synthesize) # torchaudio uses a natural logarithm
        torchaudio_spectrogram_to_synthesize_db = librosa.power_to_db(torchaudio_spectrogram_to_synthesize_linear)
        audio = mel_2_audio(torchaudio_spectrogram_to_synthesize_db)

    return audio

### Example Speech

In [ ]:
# Verify the file path is correct
file_path = '/home/bosfab01/SpeakerVerificationBA/data/preprocessed/0a4b5c0f-facc-4d3b-8a41-bc9148d62d95/0_segment_0.flac'
try:
    audio_signal, sample_rate = sf.read(file_path)
except Exception as e:
    print(f"An error occurred while reading the file: {e}")
    raise

# Save original audio to a WAV file
directory = 'audios'
if not os.path.exists(directory):
    os.makedirs(directory)
output_path_raw_audio = 'audios/raw_speech.wav'
sf.write(output_path_raw_audio, audio_signal, sample_rate)

# Create time array for plotting
time = np.arange(len(audio_signal)) / sample_rate

# Print information about the audio
print("Time of last sample:", time[-1])
print("Number of samples:", len(audio_signal))
print("Sample rate:", sample_rate)
print("Duration of audio:", len(audio_signal) / sample_rate)
print("Shape of audio signal:", audio_signal.shape)
print("Type of audio signal:", type(audio_signal))
print("Data type of audio signal:", audio_signal.dtype)


# Convert the NumPy array to a PyTorch tensor
audio_tensor = torch.from_numpy(audio_signal)
print("Type of audio tensor:", type(audio_tensor))
print("Data type of audio tensor:", audio_tensor.dtype)
print("Shape of audio tensor:", audio_tensor.shape)

# Ensure the tensor is in float32 format (required for most torchaudio operations)
audio_tensor = audio_tensor.float()
print("Data type of audio tensor:", audio_tensor.dtype)

# If your array is not in batch x channels x time format, adjust accordingly
# Assuming the audio signal is single-channel and not batched:
audio_tensor = audio_tensor.unsqueeze(0)
print("Shape of audio tensor:", audio_tensor.shape)

# Now call the fbank function
fbank_features = torchaudio.compliance.kaldi.fbank(
    audio_tensor, 
    sample_frequency=sample_rate, 
    htk_compat=True, 
    use_energy=False, 
    window_type='hanning', 
    num_mel_bins=128, 
    dither=0.0, 
    frame_shift=10
)

# Output the shape of the fbank features to confirm
print(f"Shape of fbank features: {fbank_features.shape}")

# normalize fbank features
dataset_mean=-5.0716844 
dataset_std=4.386603
fbank_features = (fbank_features - dataset_mean) / (2 * dataset_std)

# add batch dimension
fbank_features = fbank_features.unsqueeze(0)
print(f"Shape of fbank features: {fbank_features.shape}")


model = ASTModel(fshape=128, tshape=2, fstride=128, tstride=2, input_fdim=128, input_tdim=998, model_size='base', pretrain_stage=True)
model = torch.nn.DataParallel(model)
model.load_state_dict(torch.load('../pretraining/exp/pretrained-20240501-162648-original-base-f128-t2-b48-lr1e-4-m390-pretrain_joint-asli/models/audio_model.108.pth'))
model = model.module
model.to('cpu')
model.eval()
print(next(model.parameters()).device)  # Should print 'cpu'


hop_width = 20
hop_length = 50 # just for visualization, not the actual hop length used in the data preparation
hops = range(hop_length, 998//2 - hop_width//2, hop_length)
print("hops: ", hops) # range(50, 489, 50)
mask_indices = [range(i-hop_width//2, i + hop_width//2) for i in hops]
mask_indices = [idx for group in mask_indices for idx in group]
print("mask_indices: ", mask_indices) # [40, 41, 42, ..., 59, 90, 91, 92, ..., 109, 140, 141, 142, ..., 159, 190, 191, 192, ......, 459]

# turn indices from model basis [0, 499] to spectrogram basis [0, 998]
expanded_mask_indices = []
for idx in mask_indices:
    expanded_mask_indices.extend([2 * idx, 2 * idx + 1])  # Expanding indice

# Create a mask for the spectrogram
mask = torch.ones_like(fbank_features)
for idx in expanded_mask_indices:
    mask[0, idx, :] = 0  # Set the specific patches to 0

# Apply the mask to the input spectrogram
masked_spectrogram = fbank_features * mask

# turn into tensor
mask_indices = torch.tensor(mask_indices)

# Call the model
with torch.no_grad():
    reconstructed_spectrogram = model(fbank_features, task='visualize_mask', mask_indices=mask_indices)

# compare input and output
print(fbank_features.shape)
print(reconstructed_spectrogram.shape)

n_timesteps = fbank_features.shape[1]  # 998
time_per_step = 10 / 1000  # Example: if each step represents 10 ms (adjust based on your actual data setup)

# Creating time labels for every 100 steps (1 second if each step is 10 ms)
# round timesteps to next 100 and get the range
x_ticks = np.arange(0, n_timesteps, 100)
last_tick = int(np.ceil(n_timesteps / 100) * 100)
x_ticks = np.append(x_ticks, last_tick)
 # formatting time labels as strings in seconds
x_labels = [f"{x * time_per_step:.1f}s" for x in x_ticks]
x_ticks[-1] = n_timesteps

print("x_ticks:", x_ticks)
print("x_labels:", x_labels)


# y-ticks for the frequency axis
y_ticks = np.arange(0, 128, 25)

# Now plotting all three: original input, masked input, and reconstructed output
plt.figure(figsize=(12, 6))

# Define the patch width and height
patch_height = fbank_features.shape[2]  # This is the vertical height of your spectrogram

# create numpy arrays
masked_spectrogram = masked_spectrogram[0].cpu().numpy().T
reconstructed_spectrogram = reconstructed_spectrogram[0].cpu().numpy().T
fbank_features = fbank_features[0].cpu().numpy().T

# unnormalize
masked_spectrogram = masked_spectrogram * (2 * dataset_std) + dataset_mean
reconstructed_spectrogram = reconstructed_spectrogram * (2 * dataset_std) + dataset_mean
fbank_features = fbank_features * (2 * dataset_std) + dataset_mean

# print shapes
print("masked_spectrogram shape:", masked_spectrogram.shape)
print("reconstructed_spectrogram shape:", reconstructed_spectrogram.shape)
print("fbank_features shape:", fbank_features.shape)

ax1 = plt.subplot(3, 1, 1)
plt.imshow(masked_spectrogram, aspect='auto', origin='lower')
plt.title('Masked Spectrogram')
plt.xticks(x_ticks, x_labels)
plt.yticks(y_ticks)

ax2 = plt.subplot(3, 1, 2)
plt.imshow(reconstructed_spectrogram, aspect='auto', origin='lower')
plt.title('Reconstructed Spectrogram')
plt.xticks(x_ticks, x_labels)
plt.yticks(y_ticks)

ax3 = plt.subplot(3, 1, 3)
plt.imshow(fbank_features, aspect='auto', origin='lower')
plt.title('Original Spectrogram')
plt.xticks(x_ticks, x_labels) 
plt.yticks(y_ticks)

plt.tight_layout()

plt.show()


# convert back to waveform
y_masked = synthesize_audio_from_spectrogram(masked_spectrogram, reference_audio_signal=audio_signal)
y_reconstructed = synthesize_audio_from_spectrogram(reconstructed_spectrogram, reference_audio_signal=audio_signal)
y_original = synthesize_audio_from_spectrogram(fbank_features, reference_audio_signal=audio_signal)


# save audio
directory = 'audios'
if not os.path.exists(directory):
    os.makedirs(directory)
output_path_masked_audio = 'audios/masked_speech.wav'
output_path_reconstructed_audio = 'audios/reconstructed_speech.wav' 
output_path_original_audio = 'audios/original_speech.wav'
sr = 16000
sf.write(output_path_masked_audio, y_masked, sr)
sf.write(output_path_reconstructed_audio, y_reconstructed, sr)
sf.write(output_path_original_audio, y_original, sr)

In [ ]:
# compare raw and reconstructed audio spectrograms

signal_raw, sr = sf.read(output_path_raw_audio)
signal_masked, sr = sf.read(output_path_masked_audio)
signal_reconstructed, sr = sf.read(output_path_reconstructed_audio)
signal_original, sr = sf.read(output_path_original_audio)

M_raw = librosa.feature.melspectrogram(y=signal_raw, sr=sr, n_mels=128, hop_length=160, n_fft=400, center=False, htk=True, fmin=0.0, fmax=sr/2)
M_masked = librosa.feature.melspectrogram(y=signal_masked, sr=sr, n_mels=128, hop_length=160, n_fft=400, center=False, htk=True, fmin=0.0, fmax=sr/2)
M_reconstructed = librosa.feature.melspectrogram(y=signal_reconstructed, sr=sr, n_mels=128, hop_length=160, n_fft=400, center=False, htk=True, fmin=0.0, fmax=sr/2)
M_original = librosa.feature.melspectrogram(y=signal_original, sr=sr, n_mels=128, hop_length=160, n_fft=400, center=False, htk=True, fmin=0.0, fmax=sr/2)

M_raw = librosa.power_to_db(M_raw)
M_masked = librosa.power_to_db(M_masked)
M_reconstructed = librosa.power_to_db(M_reconstructed)
M_original = librosa.power_to_db(M_original)

plt.figure(figsize=(12, 8))

plt.subplot(4, 1, 1)
plt.imshow(M_masked, aspect='auto', origin='lower')
plt.title('Masked Spectrogram')

plt.subplot(4, 1, 2)
plt.imshow(M_reconstructed, aspect='auto', origin='lower')
plt.title('Reconstructed Spectrogram')

plt.subplot(4, 1, 3)
plt.imshow(M_original, aspect='auto', origin='lower')
plt.title('Original Spectrogram')

plt.subplot(4, 1, 4)
plt.imshow(M_raw, aspect='auto', origin='lower')
plt.title('Raw Spectrogram')

plt.tight_layout()

plt.show()

#### long mask

In [ ]:
# Verify the file path is correct
file_path = '/home/bosfab01/SpeakerVerificationBA/data/preprocessed/0a4b5c0f-facc-4d3b-8a41-bc9148d62d95/0_segment_0.flac'
try:
    audio_signal, sample_rate = sf.read(file_path)
except Exception as e:
    print(f"An error occurred while reading the file: {e}")
    raise

# Save original audio to a WAV file
directory = 'audios'
if not os.path.exists(directory):
    os.makedirs(directory)
output_path_raw_audio = 'audios/raw_speech.wav'
sf.write(output_path_raw_audio, audio_signal, sample_rate)

# Create time array for plotting
time = np.arange(len(audio_signal)) / sample_rate

# Print information about the audio
print("Time of last sample:", time[-1])
print("Number of samples:", len(audio_signal))
print("Sample rate:", sample_rate)
print("Duration of audio:", len(audio_signal) / sample_rate)
print("Shape of audio signal:", audio_signal.shape)
print("Type of audio signal:", type(audio_signal))
print("Data type of audio signal:", audio_signal.dtype)


# Convert the NumPy array to a PyTorch tensor
audio_tensor = torch.from_numpy(audio_signal)
print("Type of audio tensor:", type(audio_tensor))
print("Data type of audio tensor:", audio_tensor.dtype)
print("Shape of audio tensor:", audio_tensor.shape)

# Ensure the tensor is in float32 format (required for most torchaudio operations)
audio_tensor = audio_tensor.float()
print("Data type of audio tensor:", audio_tensor.dtype)

# If your array is not in batch x channels x time format, adjust accordingly
# Assuming the audio signal is single-channel and not batched:
audio_tensor = audio_tensor.unsqueeze(0)
print("Shape of audio tensor:", audio_tensor.shape)

# Now call the fbank function
fbank_features = torchaudio.compliance.kaldi.fbank(
    audio_tensor, 
    sample_frequency=sample_rate, 
    htk_compat=True, 
    use_energy=False, 
    window_type='hanning', 
    num_mel_bins=128, 
    dither=0.0, 
    frame_shift=10
)

# Output the shape of the fbank features to confirm
print(f"Shape of fbank features: {fbank_features.shape}")

# normalize fbank features
dataset_mean=-5.0716844 
dataset_std=4.386603
fbank_features = (fbank_features - dataset_mean) / (2 * dataset_std)

# add batch dimension
fbank_features = fbank_features.unsqueeze(0)
print(f"Shape of fbank features: {fbank_features.shape}")


model1 = ASTModel(fshape=128, tshape=2, fstride=128, tstride=2, input_fdim=128, input_tdim=998, model_size='base', pretrain_stage=True)
model2 = ASTModel(fshape=128, tshape=2, fstride=128, tstride=2, input_fdim=128, input_tdim=998, model_size='base', pretrain_stage=True)
model1 = torch.nn.DataParallel(model1)
model2 = torch.nn.DataParallel(model2)
model1.load_state_dict(torch.load('../pretraining/exp/pretrained-20240501-162648-original-base-f128-t2-b48-lr1e-4-m390-pretrain_joint-asli/models/audio_model.108.pth'))
model2.load_state_dict(torch.load('../pretraining/exp/pretrained-20240429-112534-shuffled-base-f128-t2-b48-lr1e-4-m390-pretrain_joint-asli/models/audio_model.54.pth'))
model1 = model1.module
model2 = model2.module
model1.to('cpu')
model2.to('cpu')
model1.eval()
model2.eval()

hop_width = 80
hop_length = 250 # just for visualization, not the actual hop length used in the data preparation
hops = range(hop_length, 998//2 - hop_width//2, hop_length)
print("hops: ", hops) # range(50, 489, 50)
mask_indices = [range(i-hop_width//2, i + hop_width//2) for i in hops]
mask_indices = [idx for group in mask_indices for idx in group]
print("mask_indices: ", mask_indices) # [40, 41, 42, ..., 59, 90, 91, 92, ..., 109, 140, 141, 142, ..., 159, 190, 191, 192, ......, 459]

# turn indices from model basis [0, 499] to spectrogram basis [0, 998]
expanded_mask_indices = []
for idx in mask_indices:
    expanded_mask_indices.extend([2 * idx, 2 * idx + 1])  # Expanding indice

# Create a mask for the spectrogram
mask = torch.ones_like(fbank_features)
for idx in expanded_mask_indices:
    mask[0, idx, :] = 0  # Set the specific patches to 0

# Apply the mask to the input spectrogram
masked_spectrogram = fbank_features * mask

# turn into tensor
mask_indices = torch.tensor(mask_indices)

# Call the model
with torch.no_grad():
    reconstructed_spectrogram1 = model1(fbank_features, task='visualize_mask', mask_indices=mask_indices)
    reconstructed_spectrogram2 = model2(fbank_features, task='visualize_mask', mask_indices=mask_indices)


# compare input and output
print(fbank_features.shape)
print(reconstructed_spectrogram.shape)

n_timesteps = fbank_features.shape[1]  # 998
time_per_step = 10 / 1000  # Example: if each step represents 10 ms (adjust based on your actual data setup)

# Creating time labels for every 100 steps (1 second if each step is 10 ms)
# round timesteps to next 100 and get the range
x_ticks = np.arange(0, n_timesteps, 100)
last_tick = int(np.ceil(n_timesteps / 100) * 100)
x_ticks = np.append(x_ticks, last_tick)
 # formatting time labels as strings in seconds
x_labels = [f"{x * time_per_step:.1f}s" for x in x_ticks]
x_ticks[-1] = n_timesteps

print("x_ticks:", x_ticks)
print("x_labels:", x_labels)


# y-ticks for the frequency axis
y_ticks = np.arange(0, 128, 25)

# Now plotting all three: original input, masked input, and reconstructed output
plt.figure(figsize=(12, 6))

# Define the patch width and height
patch_height = fbank_features.shape[2]  # This is the vertical height of your spectrogram

# create numpy arrays
masked_spectrogram = masked_spectrogram[0].cpu().numpy().T
reconstructed_spectrogram1 = reconstructed_spectrogram1[0].cpu().numpy().T
reconstructed_spectrogram2 = reconstructed_spectrogram2[0].cpu().numpy().T
fbank_features = fbank_features[0].cpu().numpy().T

# unnormalize
masked_spectrogram = masked_spectrogram * (2 * dataset_std) + dataset_mean
reconstructed_spectrogram1 = reconstructed_spectrogram1 * (2 * dataset_std) + dataset_mean
reconstructed_spectrogram2 = reconstructed_spectrogram2 * (2 * dataset_std) + dataset_mean
fbank_features = fbank_features * (2 * dataset_std) + dataset_mean


plt.subplot(4, 1, 1)
plt.imshow(masked_spectrogram, aspect='auto', origin='lower')
plt.title('Masked Spectrogram')
plt.xticks(x_ticks, x_labels)
plt.yticks(y_ticks)

plt.subplot(4, 1, 2)
plt.imshow(reconstructed_spectrogram2, aspect='auto', origin='lower')
plt.title('Reconstructed Spectrogram from Shuffled Model')
plt.xticks(x_ticks, x_labels)
plt.yticks(y_ticks)

plt.subplot(4, 1, 3)
plt.imshow(reconstructed_spectrogram1, aspect='auto', origin='lower')
plt.title('Reconstructed Spectrogram from Original Model')
plt.xticks(x_ticks, x_labels)
plt.yticks(y_ticks)

plt.subplot(4, 1, 4)
plt.imshow(fbank_features, aspect='auto', origin='lower')
plt.title('Original Spectrogram')
plt.xticks(x_ticks, x_labels) 
plt.yticks(y_ticks)

plt.tight_layout()

plt.show()


# convert back to waveform
y_masked = synthesize_audio_from_spectrogram(masked_spectrogram, reference_audio_signal=audio_signal)
y_reconstructed1 = synthesize_audio_from_spectrogram(reconstructed_spectrogram1, reference_audio_signal=audio_signal)
y_reconstructed2 = synthesize_audio_from_spectrogram(reconstructed_spectrogram2, reference_audio_signal=audio_signal)
y_original = synthesize_audio_from_spectrogram(fbank_features, reference_audio_signal=audio_signal)


# save audio
directory = 'audios'
if not os.path.exists(directory):
    os.makedirs(directory)
output_path_masked_audio = 'audios/masked_speech_longmask.wav'
output_path_reconstructed_audio1 = 'audios/reconstructed_speech_OSModel_longmask.wav'
output_path_reconstructed_audio2 = 'audios/reconstructed_speech_SUModel_longmask.wav' 
output_path_original_audio = 'audios/original_speech.wav'
sr = 16000
sf.write(output_path_masked_audio, y_masked, sr)
sf.write(output_path_reconstructed_audio1, y_reconstructed1, sr)
sf.write(output_path_reconstructed_audio2, y_reconstructed2, sr)
sf.write(output_path_original_audio, y_original, sr)

### Example Music

In [ ]:
# Verify the file path is correct
#file_path = '/home/bosfab01/SpeakerVerificationBA/data/preprocessed/0a4b5c0f-facc-4d3b-8a41-bc9148d62d95/0_segment_0.flac'
file_path = '/home/bosfab01/SpeakerVerificationBA/data/preprocessed_eval/1a281f7d-bde7-4744-9220-7ea599b6e093/0_segment_0.flac'
try:
    audio_signal, sample_rate = sf.read(file_path)
except Exception as e:
    print(f"An error occurred while reading the file: {e}")
    raise

# Save original audio to a WAV file
directory = 'audios'
if not os.path.exists(directory):
    os.makedirs(directory)
output_path_raw_audio = 'audios/raw_music.wav'
sf.write(output_path_raw_audio, audio_signal, sample_rate)

# Create time array for plotting
time = np.arange(len(audio_signal)) / sample_rate

# Convert the NumPy array to a PyTorch tensor
audio_tensor = torch.from_numpy(audio_signal)

# Ensure the tensor is in float32 format (required for most torchaudio operations)
audio_tensor = audio_tensor.float()

# If your array is not in batch x channels x time format, adjust accordingly
# Assuming the audio signal is single-channel and not batched:
audio_tensor = audio_tensor.unsqueeze(0)

# Now call the fbank function
fbank_features = torchaudio.compliance.kaldi.fbank(
    audio_tensor, 
    sample_frequency=sample_rate, 
    htk_compat=True, 
    use_energy=False, 
    window_type='hanning', 
    num_mel_bins=128, 
    dither=0.0, 
    frame_shift=10
)

# Output the shape of the fbank features to confirm
test_input = fbank_features

# keep numpy copy for audio reconstruction
test_input_numpy = test_input.clone().detach().numpy().T
print(f"Shape of numpy fbank features: {test_input_numpy.shape}")

# normalize fbank features
dataset_mean=-5.0716844 
dataset_std=4.386603
test_input = (test_input - dataset_mean) / (2 * dataset_std)

# add batch dimension
test_input = test_input.unsqueeze(0)

width = 20
hop_length = 50 # just for visualization, not the actual hop length used in the data preparation
hops = range(hop_length, 998//2 - width//2, hop_length)
mask_indices = [range(i-width//2, i + width//2) for i in hops]
mask_indices = [idx for group in mask_indices for idx in group]
print(mask_indices)

# turn indices from model basis [0, 499] to spectrogram basis [0, 998]
expanded_mask_indices = []
for idx in mask_indices:
    expanded_mask_indices.extend([2 * idx, 2 * idx + 1])  # Expanding indice

# Create a mask for the spectrogram
mask = torch.ones_like(test_input)
for idx in expanded_mask_indices:
    mask[0, idx, :] = 0  # Set the specific patches to 0

# Apply the mask to the input spectrogram
masked_spectrogram = test_input * mask

# turn into tensor
mask_indices = torch.tensor(mask_indices)

# Assuming 'test_input' is your spectrogram tensor
n_timesteps = test_input.shape[1]  # 998
time_per_step = 10 / 1000  # Example: if each step represents 10 ms (adjust based on your actual data setup)

# Creating time labels for every 100 steps (1 second if each step is 10 ms)
# round timesteps to next 100 and get the range
x_ticks = np.arange(0, n_timesteps, 100)
last_tick = int(np.ceil(n_timesteps / 100) * 100)
x_ticks = np.append(x_ticks, last_tick)
 # formatting time labels as strings in seconds
x_labels = [f"{x * time_per_step:.1f}s" for x in x_ticks]
x_ticks[-1] = n_timesteps


# y-ticks for the frequency axis
y_ticks = np.arange(0, 128, 25)


def initialize_model():
    return ASTModel(fshape=128, tshape=2, fstride=128, tstride=2, input_fdim=128, input_tdim=998, model_size='base', pretrain_stage=True)

def load_model(checkpoint_id):
    model = initialize_model()
    model = torch.nn.DataParallel(model)
    model.load_state_dict(torch.load(f'../pretraining/exp/pretrained-20240501-162648-original-base-f128-t2-b48-lr1e-4-m390-pretrain_joint-asli/models/audio_model.{checkpoint_id}.pth'))
    model = model.module
    model.to('cpu')
    model.eval()
    return model

# Load models
models = {i: load_model(i) for i in [1, 2, 6, 40, 108]}

# Generate spectrograms
with torch.no_grad():
    reconstructed_spectrograms = {i: model(test_input, task='visualize_mask', mask_indices=mask_indices) for i, model in models.items()}

# Plotting
plt.figure(figsize=(12, 14))

def plot_spectrogram(data, title, position):         
    plt.subplot(7, 1, position)
    plt.imshow(data[0].cpu().numpy().T, aspect='auto', origin='lower')
    plt.title(title)
    plt.xticks(x_ticks, x_labels)
    plt.yticks(y_ticks)

def convert_and_save_audio(data, output_path, M_for_fitting=test_input_numpy):
    M_normalized = data[0].cpu().numpy().T
    M_unnormalized = M_normalized * (2 * dataset_std) + dataset_mean
    y = synthesize_audio_from_spectrogram(M_unnormalized, reference_audio_signal=audio_signal)
    sr = 16000
    sf.write(output_path, y, sr)

# masked spectrogram
plot_spectrogram(masked_spectrogram, 'Masked Spectrogram', 1)
convert_and_save_audio(masked_spectrogram, 'audios/masked_music.wav')

# reconstructed spectrograms
for index, checkpoint in enumerate(sorted(reconstructed_spectrograms.keys())):
    plot_spectrogram(reconstructed_spectrograms[checkpoint], f'Reconstructed Spectrogram (after {(checkpoint-1)*4}k Iterations)', index + 2)
    convert_and_save_audio(reconstructed_spectrograms[checkpoint], f'audios/reconstructed_music_{checkpoint}.wav')

# original spectrogram
plot_spectrogram(test_input, 'Original Spectrogram', 7)
convert_and_save_audio(test_input, 'audios/original_music.wav')

plt.tight_layout()

plt.show()

In [ ]:
# compare raw and reconstructed audio spectrograms

signal_raw, sr = sf.read("audios/raw_music.wav")
signal_original, sr = sf.read("audios/original_music.wav")
signal_reconstructed_194, sr = sf.read("audios/reconstructed_music_108.wav")

M_raw = librosa.feature.melspectrogram(y=signal_raw, sr=sr, n_mels=128, hop_length=160, n_fft=400, center=False, htk=True, fmin=0.0, fmax=sr/2)
M_original = librosa.feature.melspectrogram(y=signal_original, sr=sr, n_mels=128, hop_length=160, n_fft=400, center=False, htk=True, fmin=0.0, fmax=sr/2)
M_reconstructed_194 = librosa.feature.melspectrogram(y=signal_reconstructed_194, sr=sr, n_mels=128, hop_length=160, n_fft=400, center=False, htk=True, fmin=0.0, fmax=sr/2)

M_raw = librosa.power_to_db(M_raw)
M_original = librosa.power_to_db(M_original)
M_reconstructed_194 = librosa.power_to_db(M_reconstructed_194)

plt.figure(figsize=(12, 6))

plt.subplot(3, 1, 1)
plt.imshow(M_raw, aspect='auto', origin='lower')
plt.title('Raw Spectrogram')

plt.subplot(3, 1, 2)
plt.imshow(M_original, aspect='auto', origin='lower')
plt.title('Original Spectrogram')

plt.subplot(3, 1, 3)
plt.imshow(M_reconstructed_194, aspect='auto', origin='lower')
plt.title('Reconstructed Spectrogram (after 108k Iterations)')

plt.tight_layout()

plt.show()

### Example Speech but with randomly distributed masked frames

#### model trained on original spectrograms

In [ ]:
# Verify the file path is correct
file_path = '/home/bosfab01/SpeakerVerificationBA/data/preprocessed/0a4b5c0f-facc-4d3b-8a41-bc9148d62d95/0_segment_0.flac'
try:
    audio_signal, sample_rate = sf.read(file_path)
except Exception as e:
    print(f"An error occurred while reading the file: {e}")
    raise

directory = 'audios'
if not os.path.exists(directory):
    os.makedirs(directory)

# Create time array for plotting
time = np.arange(len(audio_signal)) / sample_rate

# Convert the NumPy array to a PyTorch tensor
audio_tensor = torch.from_numpy(audio_signal)

# Ensure the tensor is in float32 format (required for most torchaudio operations)
audio_tensor = audio_tensor.float()

# If your array is not in batch x channels x time format, adjust accordingly
# Assuming the audio signal is single-channel and not batched:
audio_tensor = audio_tensor.unsqueeze(0)

# Now call the fbank function
fbank_features = torchaudio.compliance.kaldi.fbank(
    audio_tensor, 
    sample_frequency=sample_rate, 
    htk_compat=True, 
    use_energy=False, 
    window_type='hanning', 
    num_mel_bins=128, 
    dither=0.0, 
    frame_shift=10
)

# normalize fbank features
dataset_mean=-5.0716844 
dataset_std=4.386603
fbank_features = (fbank_features - dataset_mean) / (2 * dataset_std)

# add batch dimension
fbank_features = fbank_features.unsqueeze(0)

model = ASTModel(fshape=128, tshape=2, fstride=128, tstride=2, input_fdim=128, input_tdim=998, model_size='base', pretrain_stage=True)
model = torch.nn.DataParallel(model)
model.load_state_dict(torch.load('../pretraining/exp/pretrained-20240501-162648-original-base-f128-t2-b48-lr1e-4-m390-pretrain_joint-asli/models/audio_model.108.pth'))
model = model.module
model.to('cpu')
model.eval()

# make 250 random indices in the range [0, 499], but using a seed for reproducibility
np.random.seed(15)
mask_indices = np.random.choice(499, 370, replace=False)

# turn indices from model basis [0, 499] to spectrogram basis [0, 998]
expanded_mask_indices = []
for idx in mask_indices:
    expanded_mask_indices.extend([2 * idx, 2 * idx + 1])  # Expanding indice

# Create a mask for the spectrogram
mask = torch.ones_like(fbank_features)
for idx in expanded_mask_indices:
    mask[0, idx, :] = 0  # Set the specific patches to 0

# Apply the mask to the input spectrogram
masked_spectrogram = fbank_features * mask

# turn into tensor
mask_indices = torch.tensor(mask_indices)

# Call the model
with torch.no_grad():
    reconstructed_spectrogram = model(fbank_features, task='visualize_mask', mask_indices=mask_indices)

# compare input and output
print(fbank_features.shape)
print(reconstructed_spectrogram.shape)

n_timesteps = fbank_features.shape[1]  # 998
time_per_step = 10 / 1000  # Example: if each step represents 10 ms (adjust based on your actual data setup)

# Creating time labels for every 100 steps (1 second if each step is 10 ms)
# round timesteps to next 100 and get the range
x_ticks = np.arange(0, n_timesteps, 100)
last_tick = int(np.ceil(n_timesteps / 100) * 100)
x_ticks = np.append(x_ticks, last_tick)
 # formatting time labels as strings in seconds
x_labels = [f"{x * time_per_step:.1f}s" for x in x_ticks]
x_ticks[-1] = n_timesteps

print("x_ticks:", x_ticks)
print("x_labels:", x_labels)


# y-ticks for the frequency axis
y_ticks = np.arange(0, 128, 25)

# Now plotting all three: original input, masked input, and reconstructed output
plt.figure(figsize=(12, 6))

# Define the patch width and height
patch_height = fbank_features.shape[2]  # This is the vertical height of your spectrogram

# create numpy arrays
masked_spectrogram = masked_spectrogram[0].cpu().numpy().T
reconstructed_spectrogram = reconstructed_spectrogram[0].cpu().numpy().T
fbank_features = fbank_features[0].cpu().numpy().T

# unnormalize
masked_spectrogram = masked_spectrogram * (2 * dataset_std) + dataset_mean
reconstructed_spectrogram = reconstructed_spectrogram * (2 * dataset_std) + dataset_mean
fbank_features = fbank_features * (2 * dataset_std) + dataset_mean

# print shapes
print("masked_spectrogram shape:", masked_spectrogram.shape)
print("reconstructed_spectrogram shape:", reconstructed_spectrogram.shape)
print("fbank_features shape:", fbank_features.shape)

ax1 = plt.subplot(3, 1, 1)
plt.imshow(masked_spectrogram, aspect='auto', origin='lower')
plt.title('Masked Spectrogram')
plt.xticks(x_ticks, x_labels)
plt.yticks(y_ticks)

ax2 = plt.subplot(3, 1, 2)
plt.imshow(reconstructed_spectrogram, aspect='auto', origin='lower')
plt.title('Reconstructed Spectrogram')
plt.xticks(x_ticks, x_labels)
plt.yticks(y_ticks)

ax3 = plt.subplot(3, 1, 3)
plt.imshow(fbank_features, aspect='auto', origin='lower')
plt.title('Original Spectrogram')
plt.xticks(x_ticks, x_labels) 
plt.yticks(y_ticks)

plt.tight_layout()

plt.show()


# convert back to waveform
y_masked = synthesize_audio_from_spectrogram(masked_spectrogram, reference_audio_signal=audio_signal)
y_reconstructed = synthesize_audio_from_spectrogram(reconstructed_spectrogram, reference_audio_signal=audio_signal)
y_original = synthesize_audio_from_spectrogram(fbank_features, reference_audio_signal=audio_signal)


# save audio
directory = 'audios'
if not os.path.exists(directory):
    os.makedirs(directory)
output_path_masked_audio = 'audios/masked_speech_random_masks_originalModel.wav'
output_path_reconstructed_audio = 'audios/reconstructed_speech_random_masks_originalModel.wav' 
output_path_original_audio = 'audios/original_speech_random_masks_originalModel.wav'
sr = 16000
sf.write(output_path_masked_audio, y_masked, sr)
sf.write(output_path_reconstructed_audio, y_reconstructed, sr)
sf.write(output_path_original_audio, y_original, sr)

#### model trained on shuffled spectrograms

In [ ]:
# Verify the file path is correct
file_path = '/home/bosfab01/SpeakerVerificationBA/data/preprocessed/0a4b5c0f-facc-4d3b-8a41-bc9148d62d95/0_segment_0.flac'
try:
    audio_signal, sample_rate = sf.read(file_path)
except Exception as e:
    print(f"An error occurred while reading the file: {e}")
    raise

directory = 'audios'
if not os.path.exists(directory):
    os.makedirs(directory)

# Create time array for plotting
time = np.arange(len(audio_signal)) / sample_rate

# Convert the NumPy array to a PyTorch tensor
audio_tensor = torch.from_numpy(audio_signal)

# Ensure the tensor is in float32 format (required for most torchaudio operations)
audio_tensor = audio_tensor.float()

# If your array is not in batch x channels x time format, adjust accordingly
# Assuming the audio signal is single-channel and not batched:
audio_tensor = audio_tensor.unsqueeze(0)

# Now call the fbank function
fbank_features = torchaudio.compliance.kaldi.fbank(
    audio_tensor, 
    sample_frequency=sample_rate, 
    htk_compat=True, 
    use_energy=False, 
    window_type='hanning', 
    num_mel_bins=128, 
    dither=0.0, 
    frame_shift=10
)

# normalize fbank features
dataset_mean=-5.0716844 
dataset_std=4.386603
fbank_features = (fbank_features - dataset_mean) / (2 * dataset_std)

# add batch dimension
fbank_features = fbank_features.unsqueeze(0)

model = ASTModel(fshape=128, tshape=2, fstride=128, tstride=2, input_fdim=128, input_tdim=998, model_size='base', pretrain_stage=True)
model = torch.nn.DataParallel(model)
model.load_state_dict(torch.load('../pretraining/exp/pretrained-20240429-112534-shuffled-base-f128-t2-b48-lr1e-4-m390-pretrain_joint-asli/models/audio_model.54.pth'))
model = model.module
model.to('cpu')
model.eval()

# make 250 random indices in the range [0, 499], but using a seed for reproducibility
np.random.seed(15)
mask_indices = np.random.choice(499, 370, replace=False)

# turn indices from model basis [0, 499] to spectrogram basis [0, 998]
expanded_mask_indices = []
for idx in mask_indices:
    expanded_mask_indices.extend([2 * idx, 2 * idx + 1])  # Expanding indice

# Create a mask for the spectrogram
mask = torch.ones_like(fbank_features)
for idx in expanded_mask_indices:
    mask[0, idx, :] = 0  # Set the specific patches to 0

# Apply the mask to the input spectrogram
masked_spectrogram = fbank_features * mask

# turn into tensor
mask_indices = torch.tensor(mask_indices)

# Call the model
with torch.no_grad():
    reconstructed_spectrogram = model(fbank_features, task='visualize_mask', mask_indices=mask_indices)

# compare input and output
print(fbank_features.shape)
print(reconstructed_spectrogram.shape)

n_timesteps = fbank_features.shape[1]  # 998
time_per_step = 10 / 1000  # Example: if each step represents 10 ms (adjust based on your actual data setup)

# Creating time labels for every 100 steps (1 second if each step is 10 ms)
# round timesteps to next 100 and get the range
x_ticks = np.arange(0, n_timesteps, 100)
last_tick = int(np.ceil(n_timesteps / 100) * 100)
x_ticks = np.append(x_ticks, last_tick)
 # formatting time labels as strings in seconds
x_labels = [f"{x * time_per_step:.1f}s" for x in x_ticks]
x_ticks[-1] = n_timesteps

print("x_ticks:", x_ticks)
print("x_labels:", x_labels)


# y-ticks for the frequency axis
y_ticks = np.arange(0, 128, 25)

# Now plotting all three: original input, masked input, and reconstructed output
plt.figure(figsize=(12, 6))

# Define the patch width and height
patch_height = fbank_features.shape[2]  # This is the vertical height of your spectrogram

# create numpy arrays
masked_spectrogram = masked_spectrogram[0].cpu().numpy().T
reconstructed_spectrogram = reconstructed_spectrogram[0].cpu().numpy().T
fbank_features = fbank_features[0].cpu().numpy().T

# unnormalize
masked_spectrogram = masked_spectrogram * (2 * dataset_std) + dataset_mean
reconstructed_spectrogram = reconstructed_spectrogram * (2 * dataset_std) + dataset_mean
fbank_features = fbank_features * (2 * dataset_std) + dataset_mean

# print shapes
print("masked_spectrogram shape:", masked_spectrogram.shape)
print("reconstructed_spectrogram shape:", reconstructed_spectrogram.shape)
print("fbank_features shape:", fbank_features.shape)

ax1 = plt.subplot(3, 1, 1)
plt.imshow(masked_spectrogram, aspect='auto', origin='lower')
plt.title('Masked Spectrogram')
plt.xticks(x_ticks, x_labels)
plt.yticks(y_ticks)

ax2 = plt.subplot(3, 1, 2)
plt.imshow(reconstructed_spectrogram, aspect='auto', origin='lower')
plt.title('Reconstructed Spectrogram')
plt.xticks(x_ticks, x_labels)
plt.yticks(y_ticks)

ax3 = plt.subplot(3, 1, 3)
plt.imshow(fbank_features, aspect='auto', origin='lower')
plt.title('Original Spectrogram')
plt.xticks(x_ticks, x_labels) 
plt.yticks(y_ticks)

plt.tight_layout()

plt.show()


# convert back to waveform
y_masked = synthesize_audio_from_spectrogram(masked_spectrogram, reference_audio_signal=audio_signal)
y_reconstructed = synthesize_audio_from_spectrogram(reconstructed_spectrogram, reference_audio_signal=audio_signal)
y_original = synthesize_audio_from_spectrogram(fbank_features, reference_audio_signal=audio_signal)


# save audio
directory = 'audios'
if not os.path.exists(directory):
    os.makedirs(directory)
output_path_masked_audio = 'audios/masked_speech_random_masks_shuffledModel.wav'
output_path_reconstructed_audio = 'audios/reconstructed_speech_random_masks_shuffledModel.wav' 
output_path_original_audio = 'audios/original_speech_random_masks_shuffledModel.wav'
sr = 16000
sf.write(output_path_masked_audio, y_masked, sr)
sf.write(output_path_reconstructed_audio, y_reconstructed, sr)
sf.write(output_path_original_audio, y_original, sr)